In [1]:
import torch
from torch.utils.data import Dataset, DataLoader


In [5]:
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)
        for i in range (0, len(token_ids) - max_length, chunks):
            input_chunk = token_ids[i: i+max_length]
            output_chunk = token_ids[i+1: i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(output_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
    

In [ ]:
def create_dataloader_v1(txt, batch_size = 4, max_length = 256, )